In [18]:

#import potrebnih biblioteka
%matplotlib inline
import cv2 #OpenCV biblioteka
import collections
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD

import matplotlib.pyplot as plt
import numpy as np
import scipy as sc  #dodatna biblioteka za naucne programe
from sklearn.cluster import KMeans

import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 12 # za prikaz većih slika i plotova, zakomentarisati ako nije potrebno

import os, tkFileDialog
import shutil
import tkMessageBox as mbox
from Tkinter import *
from platform import python_version
from PIL import ImageTk
import PIL
from PIL import Image
        

#funkcije za rad sa slikama
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
def image_bin(image_gs):
    ret,image_bin = cv2.threshold(image_gs, 127, 255, cv2.THRESH_BINARY)
    return image_bin
def invert(image):
    return 255-image
def display_image(image, color= False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')
def dilate(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.dilate(image, kernel, iterations=1)
def erode(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.erode(image, kernel, iterations=1)

#Funkcionalnost implementirana u V2
def resize_region(region):
    resized = cv2.resize(region,(28,28), interpolation = cv2.INTER_NEAREST)
    return resized
def scale_to_range(image):
    return image / 255
def matrix_to_vector(image):
    return image.flatten()
def prepare_for_ann(regions):
    ready_for_ann = []
    for region in regions:
        ready_for_ann.append(matrix_to_vector(scale_to_range(region)))
    return ready_for_ann
def convert_output(outputs):
    return np.eye(len(outputs))
def winner(output):
    return max(enumerate(output), key=lambda x: x[1])[0]

# Uklanjanje šuma
def remove_noise(binary_image):
    ret_val = erode(dilate(binary_image))
    ret_val = invert(ret_val)
    return ret_val

proveri=False 

#selektovanje regiona
def select_roi(image_orig, image_bin):
    img, contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #Način određivanja kontura je promenjen na spoljašnje konture: cv2.RETR_EXTERNAL
    regions_dict = {}
    for contour in contours: 
        x,y,w,h = cv2.boundingRect(contour)
        region = image_bin[y:y+h+1,x:x+w+1];
        # Proširiti regions_dict elemente sa vrednostima boundingRect-a ili samim konturama
        regions_dict[x] = [resize_region(region), (x,y,w,h)]
        cv2.rectangle(image_orig,(x,y),(x+w,y+h),(0,255,0),2)

    sorted_regions_dict = collections.OrderedDict(sorted(regions_dict.items()))
    sorted_regions = np.array(sorted_regions_dict.values())

    sorted_rectangles = sorted_regions[:,1]
    region_distances = [-sorted_rectangles[0][0]-sorted_rectangles[0][2]]
    # Izdvojiti sortirane parametre opisujućih pravougaonika
    # Izračunati rastojanja između svih susednih regiona po x osi i dodati ih u region_distances niz
    for x,y,w,h in sorted_regions[1:-1, 1]:
        region_distances[-1] += x
        region_distances.append(-x-w)
    region_distances[-1] += sorted_rectangles[-1][0]

    return image_orig, sorted_regions[:, 0], region_distances

def create_ann():
    '''
    Implementirati veštačku neuronsku mrežu sa 28x28 ulaznih neurona i jednim skrivenim slojem od 128 neurona.
    Odrediti broj izlaznih neurona. Aktivaciona funkcija je sigmoid.
    '''
    ann = Sequential()
    # Postaviti slojeve neurona mreže 'ann'
    ann.add(Dense(128, input_dim=784, activation='sigmoid'))
    ann.add(Dense(14, activation='sigmoid'))
    return ann

def train_ann(ann, X_train, y_train):
    X_train = np.array(X_train, np.float32)
    y_train = np.array(y_train, np.float32)

    # definisanje parametra algoritma za obucavanje
    sgd = SGD(lr=0.01, momentum=0.9)
    ann.compile(loss='mean_squared_error', optimizer=sgd)

    # obucavanje neuronske mreze
    ann.fit(X_train, y_train, nb_epoch=500, batch_size=1, verbose = 0, shuffle=False, show_accuracy = False) 

    return ann

def rotate_regions(contours,angles,centers,sizes):
    '''Funkcija koja vrši rotiranje regiona oko njihovih centralnih tačaka
    Args:
        contours: skup svih kontura [kontura1, kontura2, ..., konturaN]
        angles:   skup svih uglova nagiba kontura [nagib1, nagib2, ..., nagibN]
        centers:  skup svih centara minimalnih pravougaonika koji su opisani 
                  oko kontura [centar1, centar2, ..., centarN]
        sizes:    skup parova (height,width) koji predstavljaju duzine stranica minimalnog
                  pravougaonika koji je opisan oko konture [(h1,w1), (h2,w2), ...,(hN,wN)]
    Return:
        ret_val: rotirane konture'''
    ret_val = []
    for idx, contour in enumerate(contours):

        angle = angles[idx]
        cx,cy = centers[idx]
        height, width = sizes[idx]
        if width<height:
            angle+=90

        # Rotiranje svake tačke regiona oko centra rotacije
        alpha = np.pi/2 - abs(np.radians(angle))
        region_points_rotated = np.ndarray((len(contour), 2), dtype=np.int16)
        for i, point in enumerate(contour):
            x = point[0]
            y = point[1]

            #Računa koordinate tačke nakon rotacije
            rx = np.sin(alpha)*(x-cx) - np.cos(alpha)*(y-cy) + cx
            ry = np.cos(alpha)*(x-cx) + np.sin(alpha)*(y-cy) + cy


            region_points_rotated[i] = [rx,ry]
        ret_val.append(region_points_rotated)


    return ret_val

# TODO 2
def merge_regions(contours):
    '''Funkcija koja vrši spajanje kukica i kvačica sa osnovnim karakterima
    Args:
        contours: skup svih kontura (kontura - niz tacaka bele boje)
    Return:
        ret_val: skup kontura sa spojenim kukicama i kvacicama'''
    ret_val = []
    merged_index = [] #lista indeksa kontura koje su već spojene sa nekim

    for i,contour1 in enumerate(contours): #slova
        if i in merged_index:
            continue
        min_x1 = min(contour1[:,0])
        max_x1 = max(contour1[:,0])
        min_y1 = min(contour1[:,1])
        max_y1 = max(contour1[:,1])
        for j,contour2 in enumerate(contours): #kukice
            if j in merged_index or i == j:
                continue
            min_x2 = min(contour2[:,0])
            max_x2 = max(contour2[:,0])
            min_y2 = min(contour2[:,1])
            max_y2 = max(contour2[:,1])

            #Spajanje kukica iznad slova
            #spajanje dva niza je moguće obaviti funkcijom np.concatenate((contour1,contour2))

            if len(contour1)/2>len(contour2): #provera pretpostavke da je contour1 slovo

                if (min_y1-max_y2)<max(max_y1-min_y1,max_y2-min_y2)/2 \
                and (min_x2>min_x1-5 and max_x2<max_x1+5):
                    #spajanje kontura
                    ret_val.append(np.concatenate((contour1,contour2)))
                    merged_index.append(i)
                    merged_index.append(j)


    #svi regioni koji se nisu ni sa kim spojili idu u listu kontura, bez spajanja
    for idx,contour in enumerate(contours):
        if idx not in merged_index:
            ret_val.append(contour)

    return ret_val

def display_result(outputs, alphabet, k_means):
    '''
    Funkcija određuje koja od grupa predstavlja razmak između reči, a koja između slova, i na osnovu
    toga formira string od elemenata pronađenih sa slike.
    Args:
        outputs: niz izlaza iz neuronske mreže.
        alphabet: niz karaktera koje je potrebno prepoznati
        kmeans: obučen kmeans objekat
    Return:
        Vraća formatiran string
    '''
    # Odrediti indeks grupe koja odgovara rastojanju između reči, pomoću vrednosti iz k_means.cluster_centers_
    w_space_group = max(enumerate(k_means.cluster_centers_), key = lambda x: x[1])[0]
    result = alphabet[winner(outputs[0])]
    for idx, output in enumerate(outputs[1:,:]):
        # Iterativno dodavati prepoznate elemente kao u vežbi 2, alphabet[winner(output)]
        # Dodati space karakter u slučaju da odgovarajuće rastojanje između dva slova odgovara razmaku između reči.
        # U ovu svrhu, koristiti atribut niz k_means.labels_ koji sadrži sortirana rastojanja između susednih slova.
        #if (k_means.labels_[idx] == w_space_group):
         #   result += ' '
        result += alphabet[winner(output)]
    return result

def select_roi_for_neuron(image_orig, image_bin):
    '''Oznaciti regione od interesa na originalnoj slici. (ROI = regions of interest)
        Za svaki region napraviti posebnu sliku dimenzija 28 x 28. 
        Za označavanje regiona koristiti metodu cv2.boundingRect(contour).
        Kao povratnu vrednost vratiti originalnu sliku na kojoj su obeleženi regioni
        i niz slika koje predstavljaju regione sortirane po rastućoj vrednosti x ose
    '''
    img, contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = [] # lista sortiranih regiona po x osi (sa leva na desno)
    regions_dic = {}
    for contour in contours: 
        x,y,w,h = cv2.boundingRect(contour) #koordinate i velicina granicnog pravougaonika
        area = cv2.contourArea(contour)
        if area > 150 and 1< h < 200 and w > 86:
            # kopirati [y:y+h+1, x:x+w+1] sa binarne slike i smestiti u novu sliku
            # označiti region pravougaonikom na originalnoj slici (image_orig) sa rectangle funkcijom
            region = image_bin[y:y+h+1,x:x+w+1];
            regions_dic[x] = resize_region(region)       
            cv2.rectangle(image_orig,(x,y),(x+w,y+h),(0,255,0),2)

    sorted_regions_dic = collections.OrderedDict(sorted(regions_dic.items()))
    sorted_regions = sorted_regions_dic.values()
    # sortirati sve regione po x osi (sa leva na desno) i smestiti u promenljivu sorted_regions
    return image_orig, sorted_regions
    

    
#funkcija za ucitavanje alfabeta
def ucitavanjeAlfabeta():
    global letters
    image_color=load_image(imageAlphabetPath)
    #display_image(image_color)
    # Convert to grayscale and apply Gaussian filtering
    im_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)
    im_gray = cv2.GaussianBlur(im_gray, (5, 5), 0)

    height, width = im_gray.shape
    image_bin = np.ndarray((height, width), dtype=np.uint8)

    # Threshold the image
    #ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
    im_th=cv2.adaptiveThreshold(im_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 20)


    kernele = np.ones((3,3)) # strukturni element 3x3 blok
    kerneld = np.ones((3,3))
    kernele2 = np.ones((6,6)) # strukturni element 3x3 blok

    im_th=cv2.erode(im_th, kernele, iterations=1)
    im_th=cv2.dilate(im_th, kerneld, iterations=1)
    im_th=cv2.erode(im_th, kernele2, iterations=1)
    #im_th=cv2.erode(im_th, kernele2, iterations=1)
    im_th=cv2.dilate(im_th, kerneld, iterations=1)
    im_th=cv2.erode(im_th, kernele2, iterations=1)
    im_th=cv2.dilate(im_th, kerneld, iterations=1)
    im_th=cv2.dilate(im_th, kerneld, iterations=1)
    im_th=invert(im_th)
    display_image(im_th)

    selected_regions, letters, region_distances = select_roi(image_color.copy(), im_th)
    display_image(selected_regions)
    print 'Broj prepoznatih regiona:', len(letters)
    
def Alfabet():
    #alfabet
    global ann
    global alphabet
    alphabet = ['0','1','2','3','4','5','6','7','8','9','+','-','/','*']
    inputs = prepare_for_ann(letters)
    outputs = convert_output(alphabet)
    ann = create_ann()
    ann = train_ann(ann, inputs, outputs)
    mbox.showinfo("Info","Obucavanje je zavrseno.")
   
#ucitavanje 
def ucitavanjeSlike(path):
    global rezultat
    #prepoznavanje test slike
    image_test=load_image(path)
    #display_image(image_color)
    # Convert to grayscale and apply Gaussian filtering
    im_gray2 = cv2.cvtColor(image_test, cv2.COLOR_BGR2GRAY)
    im_gray2 = cv2.GaussianBlur(im_gray2, (5, 5), 0)

    height, width = im_gray2.shape
    image_bin2 = np.ndarray((height, width), dtype=np.uint8)

    # Threshold the image
    #ret, im_th = cv2.threshold(im_gray, 90, 255, cv2.THRESH_BINARY_INV)
    im_th2=cv2.adaptiveThreshold(im_gray2, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 35, 20)


    kernele = np.ones((3,3)) # strukturni element 3x3 blok
    kerneld = np.ones((3,3))
    kernele2 = np.ones((6,6)) # strukturni element 3x3 blok

    im_th2=cv2.erode(im_th2, kernele, iterations=1)
    im_th2=cv2.dilate(im_th2, kerneld, iterations=1)
    im_th2=cv2.erode(im_th2, kernele2, iterations=1)
    #im_th=cv2.erode(im_th, kernele2, iterations=1)
    im_th2=cv2.dilate(im_th2, kerneld, iterations=1)
    im_th2=cv2.erode(im_th2, kernele2, iterations=1)
    im_th2=cv2.dilate(im_th2, kerneld, iterations=1)
    im_th2=cv2.dilate(im_th2, kerneld, iterations=1)
    im_th2=cv2.erode(im_th2, kernele2, iterations=2)
    im_th2=invert(im_th2)
    display_image(im_th2)

    selected_regions2, letters2, region_distances2 = select_roi(image_test.copy(), im_th2)
    #display_image(selected_regions2)
    print 'Broj prepoznatih regiona:', len(letters2)
    
    region_distances2 = np.array(region_distances2).reshape(len(region_distances2), 1)
    k_means_test = KMeans(n_clusters=2, max_iter=200000, tol=0.00001, n_init=16)
    k_means_test.fit(region_distances2)
    inputs_test = prepare_for_ann(letters2)
    results_test = ann.predict(np.array(inputs_test, np.float32))
    print display_result(results_test, alphabet, k_means_test) 
    rezultat=display_result(results_test, alphabet, k_means_test) 
    print "Izraz sa slike: "+rezultat
    E1.delete(0, END)
    E1.insert(0,rezultat)
    
def izracunajRez():
    import wolframalpha
    client = wolframalpha.Client('GXV5GX-22UK9WJXHP')
    res = client.query(rezultat)
    print "REZULTAT:"+res.pods[1].text
    E1.insert(len(rezultat),"="+res.pods[1].text)

def imageLoad(): 
    global imagePath 
    global image
    global photo
    global proveri
    global canvas
    
    imageDir = tkFileDialog.askopenfilename()
    if imageDir == None:
        return
    
    
    imageSplit = imageDir.split("/")
    #duzina je 7 +str(len(image))
    imagePath = imageSplit[len(imageSplit)-2]+"/"+imageSplit[len(imageSplit)-1]
    proveri = True
    
    image = Image.open(imagePath)
    image = image.resize((400, 300), Image.ANTIALIAS)
    photo = ImageTk.PhotoImage(image)
    
    canvas.pack_forget()
    canvas = Canvas(root, width=image.size[0], height=image.size[1])
    canvas.create_image(0, 0, anchor=NW, image=photo)
    canvas.pack()
    '''canvas.config(yscrollcommand=scrollbar_vert.set)
    canvas.config(xscrollcommand=scrollbar_hor.set)
    canvas.config(scrollregion=canvas.bbox(ALL))
    scrollbar_vert.config(command=canvas.yview)
    scrollbar_hor.config(command=canvas.xview)
    '''
    ucitavanjeSlike(imagePath)
    btn4.config(state="normal")
    #mbox.showinfo("Info","Image Directori:"+imagePath)
    
'''def iscrtajImage():
    #if proveri==True:
    mbox.showinfo("Info","AAAAA:"+imagePath)
    image = Image.open(imagePath)
    photo = ImageTk.PhotoImage(image)
    labelImage.set = Label(image = photo)
    labelImage.pack()
'''

def alphabetLoadImage():
    global imageAlphabetPath
    global image
    global photo
    global proveri
    global canvas
    
    imageAlphabetDir = tkFileDialog.askopenfilename()
    if imageAlphabetDir == None:
        return
    
    imageAlphabetSplit = imageAlphabetDir.split("/")
    #duzina je 7 +str(len(image))
    imageAlphabetPath = imageAlphabetSplit[len(imageAlphabetSplit)-2]+"/"+imageAlphabetSplit[len(imageAlphabetSplit)-1]
    print "PUTANJA ALPHABETA JE:" + imageAlphabetPath
    proveri = True
    
    image = Image.open(imageAlphabetPath)
    image = image.resize((400, 300), Image.ANTIALIAS)
    photo = ImageTk.PhotoImage(image)
    
    canvas.pack_forget()
    canvas = Canvas(root, width=image.size[0], height=image.size[1])
    canvas.create_image(0, 0, anchor=NW, image=photo)
    canvas.pack()
    
    btn2.config(state="normal")
    

def pripremi():
    ucitavanjeAlfabeta()
    Alfabet()
    btn3.config(state="normal")
  
def quit():
    root.destroy()
    
    
root = Tk()
labelImage = Label()
labelImage.pack()

btn5 = Button(root, text='Quit', command=quit)
btn5.pack(side = BOTTOM, padx=5, pady=5)
btn4 = Button(root, text='Izracunaj', state=DISABLED,command = izracunajRez)
btn4.pack(side = BOTTOM, padx=5, pady=5)
btn3 = Button(root, text = 'Ucitaj test sliku', state=DISABLED,command = imageLoad)
btn3.pack(side = BOTTOM, padx=5,pady=5)
btn2 = Button(root, text='Obuci', state=DISABLED,command=pripremi)
btn2.pack(side = BOTTOM, padx=5, pady=5)
btn1 = Button(root, text='Ucitaj alphabet sliku', command=alphabetLoadImage)
btn1.pack(side = BOTTOM, padx=5, pady=5)


label = Label(root, text = "Rezultat")
E1 = Entry(root, bd=5)
labelImage = Label()
labelImage.pack()
label.pack()
E1.pack(pady=15)

canvas = Canvas(root,width = 400,height = 300)
canvas.pack()
'''scrollbar_vert = Scrollbar(root)
scrollbar_vert.pack(side=RIGHT, fill=Y)
scrollbar_hor = Scrollbar(root)
scrollbar_hor.config(orient=HORIZONTAL)
scrollbar_hor.pack(side=BOTTOM, fill=X) 
'''

print os.getcwd()

root.eval('tk::PlaceWindow %s center' % root.winfo_pathname(root.winfo_id()))
root.geometry('750x650')
root.title("Soft")
root.mainloop()

/home/student/Desktop/Projbackup
